In [2]:
import sys
sys.path.append('..')
import os
import json
import glob
import shutil
import pandas as pd
import numpy as np
import open3d as od
from utils.cow import get_cow_regno

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def get_from_nias_ground_truth_file(cow_regno, ground_truth_dataframe, score_column='bcs'):
    df = ground_truth_dataframe[ground_truth_dataframe['cow_regno']==cow_regno]
    if len(df) != 1:
        raise ValueError("Problem with groundtruth of this cow_regno.")
    score = df[[score_column]].values[0][0]
    return score

In [4]:
with open('./config.json', 'r') as fp:
    config = json.load(fp)

In [8]:
pcd_paths = config['pcd_paths']
json_paths = config['json_paths']
rump_path = config['rump_path']
label_score_path = config['label_score_path']
if os.path.exists(rump_path):
    shutil.rmtree(rump_path)
os.mkdir(rump_path)

In [9]:
ground_truth_dataframe = pd.read_excel("E:/ZOOTOS RnD/NIAS/Data/DataTest/ground_truth.xlsx")

In [10]:
bname = 'Rump'

In [11]:
cows, bcs_score = [], []
center_x, center_y, center_z = [], [], []
count = 0
nolabel = []
for json_path, pcd_path in zip(json_paths, pcd_paths):
    list_json = glob.glob(f'{json_path}/*.json')
    list_json = [fname for fname in list_json if not fname.endswith('classes.json')]
    for json_path in list_json:
        cow_regno = get_cow_regno(json_path)
        try:
            score = get_from_nias_ground_truth_file(cow_regno, ground_truth_dataframe, 'bcs')
        except Exception as e:
            print(e, cow_regno, json_path)
            nolabel.append(cow_regno)
            continue
        with open(json_path, 'r') as f:
            box_data = json.load(f)
        continue

        try:
            if len(box_data['objects']) == 0:
                print("Not labeled yet.")
                continue
        except:
            print("Not labeled yet")
            continue
        bboxes = {obj['name']: np.array(obj['vertices']) for obj in box_data['objects']}
        if bname not in bboxes.keys():
            print("Name error")
            continue

        pcd = od.io.read_point_cloud(f"{pcd_path}/{cow_regno}.ply")
        points = np.asarray(pcd.points)

        bbox = bboxes['Rump']
        
        p0 = bbox[0]
        p1 = bbox[3]
        p2 = bbox[1]
        p3 = bbox[4]

        # p0 as origin
        v1 = p1 - p0
        v2 = p2 - p0
        v3 = p3 - p0

        u = points - p0
        u_dot_v1 = u.dot(v1)
        u_dot_v2 = u.dot(v2)
        u_dot_v3 = u.dot(v3)

        inside_v1 = np.logical_and(np.sum(v1**2) > u_dot_v1, u_dot_v1 > 0)
        inside_v2 = np.logical_and(np.sum(v2**2) > u_dot_v2, u_dot_v2 > 0)
        inside_v3 = np.logical_and(np.sum(v3**2) > u_dot_v3, u_dot_v3 > 0)

        points_inside = np.logical_and(np.logical_and(inside_v1, inside_v2), inside_v3)
        idx = np.where(points_inside == True)[0]
        part = pcd.select_by_index(idx)
        center = np.asarray(part.points).mean(axis=0)
        part = part.translate(-center)
        x, y, z = np.asarray(part.points).mean(axis=0)
        center_x.append(x)
        center_y.append(y)
        center_z.append(z)
        od.io.write_point_cloud(f"{rump_path}/{cow_regno}.ply", part)
        cows.append(cow_regno)
        bcs_score.append(get_from_nias_ground_truth_file(cow_regno, ground_truth_dataframe, 'bcs'))
        print(f"Done: {cow_regno}")
        count += 1
print('Total rumps: ', count)

Problem with groundtruth of this cow_regno. 501127980 H:/Dairy Cow 2023/F0/Body/Cleaned/Labelling/Labelled_29231122/body_condition_score/labels\501127980.json
Problem with groundtruth of this cow_regno. 501152983 H:/Dairy Cow 2023/F0/Body/Cleaned/Labelling/Labelled_29231122/body_condition_score/labels\501152983.json
Problem with groundtruth of this cow_regno. 501165659 H:/Dairy Cow 2023/F0/Body/Cleaned/Labelling/Labelled_29231122/body_condition_score/labels\501165659.json
Problem with groundtruth of this cow_regno. 501190609 H:/Dairy Cow 2023/F0/Body/Cleaned/Labelling/Labelled_29231122/body_condition_score/labels\501190609.json
Problem with groundtruth of this cow_regno. 501190622 H:/Dairy Cow 2023/F0/Body/Cleaned/Labelling/Labelled_29231122/body_condition_score/labels\501190622.json
Problem with groundtruth of this cow_regno. 501206857 H:/Dairy Cow 2023/F0/Body/Cleaned/Labelling/Labelled_29231122/body_condition_score/labels\501206857.json
Problem with groundtruth of this cow_regno. 50

In [20]:
print(len(cows), len(bcs_score))

322 322


In [21]:
label = pd.DataFrame({'cow_regno': cows, 'bcs': bcs_score, 'center_x': center_x, 'center_y': center_y, 'center_z': center_z})
label.to_csv(label_score_path, index=False)

In [12]:
len(nolabel)

104

In [14]:
nolabel

[501127980,
 501152983,
 501165659,
 501190609,
 501190622,
 501206857,
 501243768,
 501247495,
 501247809,
 501255038,
 501255052,
 501257390,
 501260279,
 501265507,
 501267641,
 501267644,
 501267652,
 501276618,
 501278497,
 501278692,
 501292294,
 501303776,
 501303784,
 501307191,
 501309609,
 501312792,
 501312793,
 501313306,
 501327223,
 501328655,
 501329751,
 501329970,
 501329981,
 501330012,
 501330930,
 501330933,
 501335821,
 501338355,
 501343323,
 501343543,
 501350815,
 501350967,
 501350970,
 501351031,
 501351046,
 501351074,
 501352013,
 501361396,
 501363079,
 501363094,
 501363095,
 501363322,
 501366814,
 501366848,
 501366852,
 501366854,
 501370578,
 501373550,
 501373552,
 501373570,
 501373589,
 501375041,
 501375355,
 501379849,
 501379850,
 501385662,
 501385910,
 501395269,
 501396364,
 501400456,
 501402031,
 501402032,
 501402393,
 501403207,
 501408589,
 501408595,
 501408596,
 501408601,
 501408605,
 501408609,
 501408819,
 501415040,
 501420237,
 501

In [15]:
df = pd.DataFrame({'cow_regno': nolabel})
df.to_csv('nolabel.csv', index=False)

In [ ]:
"H:\Dairy Cow 2023"

In [17]:
len('김준수')

3

In [18]:
list_person = os.listdir("H:\Dairy Cow 2023")
list_person = [fname for fname in list_person if len(fname)==3]
list_person

['김준수',
 '김준호',
 '김진형',
 '문경덕',
 '박귀룡',
 '박찬혁',
 '박형석',
 '안용식',
 '윤장운',
 '이승수',
 '임요순',
 '전혁진',
 '조명현',
 '최진욱',
 '테스트',
 '황동욱']

In [22]:
nolabel

[501127980,
 501152983,
 501165659,
 501190609,
 501190622,
 501206857,
 501243768,
 501247495,
 501247809,
 501255038,
 501255052,
 501257390,
 501260279,
 501265507,
 501267641,
 501267644,
 501267652,
 501276618,
 501278497,
 501278692,
 501292294,
 501303776,
 501303784,
 501307191,
 501309609,
 501312792,
 501312793,
 501313306,
 501327223,
 501328655,
 501329751,
 501329970,
 501329981,
 501330012,
 501330930,
 501330933,
 501335821,
 501338355,
 501343323,
 501343543,
 501350815,
 501350967,
 501350970,
 501351031,
 501351046,
 501351074,
 501352013,
 501361396,
 501363079,
 501363094,
 501363095,
 501363322,
 501366814,
 501366848,
 501366852,
 501366854,
 501370578,
 501373550,
 501373552,
 501373570,
 501373589,
 501375041,
 501375355,
 501379849,
 501379850,
 501385662,
 501385910,
 501395269,
 501396364,
 501400456,
 501402031,
 501402032,
 501402393,
 501403207,
 501408589,
 501408595,
 501408596,
 501408601,
 501408605,
 501408609,
 501408819,
 501415040,
 501420237,
 501

In [27]:
list_person = os.listdir("H:/Dairy Cow 2023/Updated_Ply")
person_, farm_, cow_ = [], [], []
for person in list_person:
    list_farm = os.listdir(f"H:/Dairy Cow 2023/Updated_Ply/{person}")
    for farm in list_farm:
        list_id = os.listdir(f"H:/Dairy Cow 2023/Updated_Ply/{person}/{farm}")
        for cow in list_id:
            cow = int(cow.split('.')[0])
            if cow in nolabel:
                cow_.append(cow)
                farm_.append(farm)
                person_.append(person)

In [28]:
df = pd.DataFrame({'cow_regno': cow_, 'person': person_, 'farm': farm})
df.to_csv('nolabel.csv', index=False)

In [36]:
person_, farm_, cow_ = [], [], []
list_cows = glob.glob("H:/Dairy Cow 2023/Updated_Ply/*/*/*.ply")
for fname in list_cows:
    cow_regno = get_cow_regno(fname)
    if cow_regno in nolabel:
        person, farm = fname.split('\\')[-3:-1]
        print(farm)
        cow_.append(cow_regno)
        person_.append(person)
        farm_.append(farm)

ShinsaemRanch
ShinsaemRanch
ShinsaemRanch
ShinsaemRanch
ShinsaemRanch
TonattaRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
TopyeongRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AnsanRanch
AwonRanch
AwonRanch
AwonRanch
BorimRanch
BorimRanch
BorimRanch
BorimRanch
BorimRanch
BorimRanch
BorimRanch
CheongsolRanch
CheongsolRanch
CheongsolRanch
CheongsolRanch
CheongsolRanch
CheongsolRanch
CheongsolRanch
JeosanRanch
JeosanRanch
JeosanRanch
JeosanRanch
JeosanRanch
JeosanRanch
SeongeunRanch
JunamRanch
JunamRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
DuraeRanch
HerbRanch
HerbRanch
HerbRanch
HerbRanch
HerbRanch
SeungbeomRanch
SeungbeomRanch
Seungbe

In [37]:
df = pd.DataFrame({'cow_regno': cow_, 'person': person_, 'farm': farm_})
df.to_csv('nolabel.csv', index=False)

In [33]:
fname.split('\\')

['H:/Dairy Cow 2023/Updated_Ply',
 'YongsikAhn',
 'HaengJeongRanch',
 '501412487.ply']